<a href="https://colab.research.google.com/github/sayanbanerjee32/feedback-prize-effectiveness/blob/main/all_text_concat_ulmfit_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install fastkaggle if not available
!pip install -Uq fastai
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

# !pip install -Uq 'timm>=0.6.2.dev'
!pip install -Uq pynvml
from fastkaggle import *

     |████████████████████████████████| 46 kB 2.5 MB/s 


In [ ]:
import os
from pathlib import Path
import fastai
fastai.__version__

'2.7.9'

In [ ]:
# config depending on whether this is running on kaggle or collab
# is_colab = True
is_colab = not os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
comp = 'feedback-prize-effectiveness'
if is_colab:
    model_save_path = Path('/content/'+comp+'_out/models')
else:
    model_save_path = Path('/kaggle/working/'+comp) #+'/models')

In [ ]:
# import colab libraries
if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
if is_colab:
    !mkdir ~/.kaggle
    !cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# This permissions change avoids a warning on Kaggle tool startup.
if is_colab:
    !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
path = setup_comp(comp)
path

100%|██████████| 8.13M/8.13M [00:00<00:00, 244MB/s]

Path('feedback-prize-effectiveness')

In [ ]:
from fastai.text.all import *
set_seed(32)

## Language model for original text

In [ ]:
# In this cell fastai TextDataLoader functions are overridden to allow taking output 
# directory as input so that the toknised data is not written in data input directory
# in Kaggle, data input directory is read-only
from fastai.text.core import _tokenize_files
@delegates(_tokenize_files)
def tokenize_folderOP(path, extensions=None, folders=None,
                      output_dir=None, skip_if_exists=True, **kwargs):
    """replacement of fastai.text.core.tokenize_folder so that it passes
     output_dir as input to _tokenize_files"""
    path,extensions = Path(path),ifnone(extensions, ['.txt'])
    files = get_files(path, extensions=extensions, recurse=True, folders=folders)
    def _f(i,output_dir): return output_dir/files[i].relative_to(path)
    return _tokenize_files(_f, files, path, output_dir, 
                           skip_if_exists=skip_if_exists, **kwargs)

class TokenizerOP(Tokenizer):
    """Wrapper class for fastai.text.core.Tokenizer class to override
    from_folder method to take output_dir as input and call the overridden
    method tokenize_folderOP"""
    def __init__(self, tok, rules=None, counter=None,
                 lengths=None, mode=None, sep=' '):
        super().__init__(tok, rules=rules, counter=counter,
                 lengths=lengths, mode=mode, sep=sep)
    
    @classmethod
    @delegates(tokenize_folderOP, keep=True)
    def from_folder(cls, path, tok=None, rules=None, output_dir = None, **kwargs):
        path = Path(path)
        if tok is None: tok = WordTokenizer()
        output_dir = tokenize_folderOP(path, tok=tok, rules=rules, 
                                       skip_if_exists = False, 
                                       output_dir = output_dir, **kwargs)
        res = cls(tok, counter=load_pickle(output_dir/fn_counter_pkl),
                  lengths=load_pickle(output_dir/fn_lengths_pkl),
                  rules=rules, mode='folder')
        res.path,res.output_dir = path,output_dir
        return res


class TextBlockOP(TextBlock):
    "Overriding TextBlock for user defined output dir"
    def __init__(self, tok_tfm, vocab=None, is_lm=False,
                 seq_len=72, backwards=False, **kwargs):
        super().__init__(tok_tfm, vocab=vocab, is_lm=is_lm,
                 seq_len=seq_len, backwards=backwards, **kwargs)
    @classmethod
    @delegates(TokenizerOP.from_folder, keep=True)
    def from_folder(cls, path, vocab=None, is_lm=False, seq_len=72,
                    backwards=False, min_freq=3, max_vocab=60000,output_dir =None,
                    **kwargs):
        "Build a `TextBlock` from a `path` - calls TokenizerOP and provides output_dir as input"
        return cls(TokenizerOP.from_folder(path, output_dir = output_dir, **kwargs), vocab=vocab, is_lm=is_lm, seq_len=seq_len,
                   backwards=backwards, min_freq=min_freq, max_vocab=max_vocab)
        
class TextDataLoadersOP(TextDataLoaders):
    "Basic wrapper around several `DataLoader`s with factory methods for NLP problems"
    @classmethod
    @delegates(DataLoaders.from_dblock)
    def from_folder(cls, path, train='train', valid='valid', valid_pct=None,
                    seed=None, vocab=None, text_vocab=None, is_lm=False,
                    tok_tfm=None, seq_len=72, splitter=None,
                    backwards=False, output_dir =None, **kwargs):
        "This is to override same method from TextDataLoaders to accept and pass output_dir as input"
        if splitter is None:
            splitter = GrandparentSplitter(train_name=train,
                                           valid_name=valid) if valid_pct is None else RandomSplitter(valid_pct,
                                                                                                      seed=seed)
        blocks = [TextBlockOP.from_folder(path, text_vocab, is_lm, seq_len, backwards,
                                          tok=tok_tfm, output_dir = output_dir)]
        if not is_lm: blocks.append(CategoryBlock(vocab=vocab))
        get_items = partial(get_text_files, folders=[train,valid]) if valid_pct is None else get_text_files
        dblock = DataBlock(blocks=blocks,
                           get_items=get_items,
                           splitter=splitter,
                           get_y=None if is_lm else parent_label)
        return cls.from_dblock(dblock, path, path=path, seq_len=seq_len, **kwargs)


In [ ]:
tok_path = model_save_path / 'train_tok'
tok_path.mkdir(parents=True, exist_ok=True)
dls_lm = TextDataLoadersOP.from_folder(path / 'train', is_lm=True,
                                       valid_pct=0.1, output_dir = tok_path)
# TextDataLoaders.from_folder(path / 'train', is_lm=True, valid_pct=0.1, )
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxmaj in xxmaj german suburb , this is a "" car - free "" place , this may seem shocking to many for people who use cars to go where they need to everyday … ▁ xxmaj car ownership is only allowed in two places ; large parking garages at the edge of development , where a car owner buys a space , for $ 40 , xxrep 3 0 ,","xxmaj in xxmaj german suburb , this is a "" car - free "" place , this may seem shocking to many for people who use cars to go where they need to everyday … ▁ xxmaj car ownership is only allowed in two places ; large parking garages at the edge of development , where a car owner buys a space , for $ 40 , xxrep 3 0 , along"
1,"a home . vauban , home to 5,500 residents within a rectangular square mile , is the most advanced experiment in low car suburban life . "" all of our development since xxmaj world xxmaj war xxup ii has been centered on the car , and that will have to change , "" said david goldberg . xxbos dear , senator \n\n i think we should in take concideration in the next","home . vauban , home to 5,500 residents within a rectangular square mile , is the most advanced experiment in low car suburban life . "" all of our development since xxmaj world xxmaj war xxup ii has been centered on the car , and that will have to change , "" said david goldberg . xxbos dear , senator \n\n i think we should in take concideration in the next few"
2,"so he or she could go to the field trip . \n\n xxmaj those were my reason why is policy 1 is better than policy number 2 . xxbos xxmaj after reading this controversial essay on xxmaj electoral xxmaj college and how it affects other and our "" american xxmaj people "" i feel personally a bit ashamed at our own country that we allow all this nonsense to have happened .","he or she could go to the field trip . \n\n xxmaj those were my reason why is policy 1 is better than policy number 2 . xxbos xxmaj after reading this controversial essay on xxmaj electoral xxmaj college and how it affects other and our "" american xxmaj people "" i feel personally a bit ashamed at our own country that we allow all this nonsense to have happened . xxmaj"
3,"growing up with social problems , such as depression , or suicidal thoughts , "" . xxmaj looking at this , it shows that we are still helping people break free , and really live their lives , the others may just need more help with meeting new people . xxmaj everyone can do it , it just might take more time for others . \n\n xxmaj overall , i feel that","up with social problems , such as depression , or suicidal thoughts , "" . xxmaj looking at this , it shows that we are still helping people break free , and really live their lives , the others may just need more help with meeting new people . xxmaj everyone can do it , it just might take more time for others . \n\n xxmaj overall , i feel that i"
4,"to be a compromise between election of the xxmaj president by a vote in xxmaj congress and election of the xxmaj president by a popular vote of qualified citizens . xxmaj the xxmaj electoral xxmaj college consists of the meeting of electors where they vote for xxmaj president and xxmaj vice xxmaj president , and the counting of electoral votes by congress . \n\n xxmaj doing away with the xxmaj electoral xxmaj","be a compromise between election of the xxmaj president by a vote in xxmaj congress and election of the xxmaj president by a popular vote of qualified citizens . xxmaj the xxmaj electoral xxmaj college consists of the meeting of electors where they vote for xxmaj president and xxmaj vice xxmaj president , and the counting of electoral votes by congress . \n\n xxmaj doing away with the xxmaj electoral xxmaj xxunk"


In [ ]:
# dump vocab that will be required for inference script
with open(model_save_path / 'dls_lm_vocab.pickle', 'wb') as b:
    pickle.dump(dls_lm.vocab,b)

In [ ]:
learn_lm = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [ ]:
learn_lm.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.766937,3.447428,0.324247,31.419462,01:40


In [ ]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(10, 1e-3)

NameError: ignored

In [ ]:
model_save_path.mkdir(parents=True, exist_ok=True)
if iskaggle:
    # hack to save encoder in a writable location
    learn_lm.path = model_save_path
    learn_lm.save_encoder('finetuned_enc')
    # learn_lm.save('finetuned_lm')
else:
    learn_lm.save_encoder(model_save_path / 'finetuned_enc')

## Text classification

In [ ]:
def file_read(file_path):
    with open(file_path, 'r') as _f: 
        all_content = _f.read()
    return all_content

In [ ]:
# pre-process text - add all columns 
df = pd.read_csv(path/'train.csv')
df['essay_text'] = df['essay_id'].apply(lambda x: file_read(path / 'train' / f'{x}.txt'))
df.head()

,discourse_id,essay_id,discourse_text,discourse_type,discourse_effectiveness,essay_text
0,0013cc385424,007ACE74B050,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform.",Lead,Adequate,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform\n\nI think that the face is a natural landform because there is no li..."
1,9704a709b505,007ACE74B050,"On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform",Position,Adequate,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform\n\nI think that the face is a natural landform because there is no li..."
2,c22adee811b6,007ACE74B050,I think that the face is a natural landform because there is no life on Mars that we have descovered yet,Claim,Adequate,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform\n\nI think that the face is a natural landform because there is no li..."
3,a10d361e54e4,007ACE74B050,"If life was on Mars, we would know by now. The reason why I think it is a natural landform because, nobody live on Mars in order to create the figure. It says in paragraph 9, ""It's not easy to target Cydonia,"" in which he is saying that its not easy to know if it is a natural landform at this point. In all that they're saying, its probably a natural landform.",Evidence,Adequate,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform\n\nI think that the face is a natural landform because there is no li..."
4,db3e453ec4e2,007ACE74B050,People thought that the face was formed by alieans because they thought that there was life on Mars.,Counterclaim,Adequate,"Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture

In [ ]:
# See sequence length
df['seq_length_essay'] = [len(txt.split()) for txt in df['essay_text'].tolist()]
df['seq_length_dis'] = [len(txt.split()) for txt in df['discourse_text'].tolist()]
df['seq_length_essay'].describe(), df['seq_length_dis'].describe()

(count    36765.000000
 mean       458.588522
 std        220.423420
 min        144.000000
 25%        288.000000
 50%        408.000000
 75%        579.000000
 max       1367.000000
 Name: seq_length_essay, dtype: float64, count    36765.000000
 mean        44.654073
 std         46.669682
 min          1.000000
 25%         16.000000
 50%         28.000000
 75%         57.000000
 max        836.000000
 Name: seq_length_dis, dtype: float64)

In [ ]:
# replace discorse text within the context text as __MASKED__
df['masked_ess_txt'] = df[['essay_text','discourse_text']].apply(lambda row: row.essay_text.strip().replace(row.discourse_text.strip(),
                                                                                                         '__MASKED__'),
                                                              axis = 1)
df['seq_length_mask_ess'] = [len(txt.split()) for txt in df['masked_ess_txt'].tolist()]
df['masked_ess_txt'].head(), df['seq_length_mask_ess'].describe()

(0    __MASKED__ On my perspective, I think that the face is a natural landform because I dont think that there is any life on Mars. In these next few paragraphs, I'll be talking about how I think that is is a natural landform\n\nI think that the face is a natural landform because there is no life on Mars that we have descovered yet. If life was on Mars, we would know by now. The reason why I think it is a natural landform because, nobody live on Mars in order to create the figure. It says in paragraph 9, "It's not easy to target Cydonia," in which he is saying that its not easy to know if it is...
 1    Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. __MASKED__\n\nI think that the face is a natural landform because there

In [ ]:
# function to truncate discourse text and context text
# this is still in progress
# def trunc_text(text, num_words, unique_centre_tok = None):
#     w_l = text.split()
#     if unique_centre_tok is None:
#         if len(w_l) > num_words: w_l = w_l[:num_words]
#     else:
#         if len(w_l) > num_words:
#             try: pos_tok = w_l.index(unique_centre_tok) + 1
#             except: # in case there is an issue with the replacement
#                 print(text)
#                 pos_tok = round(num_words / 2)
#             if pos_tok > round(len(w_l) / 2):
#                 if len(w_l) > pos_tok + round((num_words - 1) / 2):
#                     start_pos = pos_tok - round((num_words - 1) / 2) - 1
#                 else:
#                     start_pos = len(w_l) - num_words
#                 w_l = w_l[start_pos:(start_pos + num_words)]
#             else:
#                 if pos_tok > round((num_words -1) / 2):
#                     start_pos = pos_tok - round((num_words -1) / 2) - 1
#                     w_l = w_l[start_pos:(start_pos + num_words)]
#                 else:
#                     w_l = w_l[:num_words]
            
#     return ' '.join(w_l)

# [trunc_text("let's see where __MASKED__ we are going.", i, unique_centre_tok = "__MASKED__") for i in range(2,8)]

In [ ]:
# combine all text columns for classification 
# concat all
# df['all_text'] = 'CONTEXT: ' + df.essay_text + '; DISCOURSE: ' + df.discourse_text + '; TYPE: ' + df.discourse_type

# concat after truncation
# df['essay_text_trunc'] = df.masked_ess_txt.apply(lambda t: trunc_text(t,512, "__MASKED__"))
# df['discourse_text_trunc'] = df.discourse_text.apply(lambda t: trunc_text(t,64))
# df['all_text'] = 'CONTEXT: ' + df.essay_text_trunc + '; TYPE: ' + df.discourse_type + '; DISCOURSE: ' + df.discourse_text_trunc 
df['all_text'] = 'CONTEXT: ' + df.masked_ess_txt + '; TYPE: ' + df.discourse_type + '; DISCOURSE: ' + df.discourse_text
df['seq_length_all'] = [len(txt.split()) for txt in df['all_text'].tolist()]
df['seq_length_all'].describe()

count    36765.000000
mean       463.536244
std        220.382475
min        149.000000
25%        293.000000
50%        413.000000
75%        584.000000
max       1373.000000
Name: seq_length_all, dtype: float64

In [ ]:
# create data loaders
dls = TextDataLoaders.from_df(df, text_col='all_text',
                              label_col='discourse_effectiveness',
                            #   seq_len=600,
                              text_vocab=dls_lm.vocab)
dls.show_batch(max_n=3)

NameError: ignored

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, 
                                drop_mult=0.5,
                                backwards=False,
                                metrics=[accuracy,F1Score(average='weighted')]).to_fp16()
# load encoder from language model
if iskaggle: model_save_path = model_save_path / 'models'
learn = learn.load_encoder(model_save_path / 'finetuned_enc')

In [ ]:
learn.lr_find(suggest_funcs=(valley, slide))

In [ ]:
learn.fine_tune(1, 0.01)

In [ ]:
model_save_path.mkdir(parents=True, exist_ok=True)
# learn.export(f'{model_save_path}/all_col_concat_learner.pkl')
learn.save(f'{model_save_path}/all_text_concat_ulmfit_save.pkl')

## Test submission

In [ ]:
test_df = pd.read_csv(path/'test.csv')
# pre-process test df texts
test_df['essay_text'] = test_df['essay_id'].apply(lambda x: file_read(path / 'test' / f'{x}.txt'))
test_df['masked_ess_txt'] = test_df[['essay_text','discourse_text']].apply(lambda row: row.essay_text.strip().replace(row.discourse_text.strip(),
                                                                                                         '__MASKED__'),
                                                              axis = 1)
# test_df['essay_text_trunc'] = test_df.masked_ess_txt.apply(lambda t: trunc_text(t,512,'__MASKED__'))
# test_df['discourse_text_trunc'] = test_df.discourse_text.apply(lambda t: trunc_text(t,64))
test_df['text'] = 'CONTEXT: ' + test_df.masked_ess_txt + '; TYPE: ' + test_df.discourse_type + '; DISCOURSE: ' + test_df.discourse_text 

In [ ]:
# test_df.rename(columns = {'discourse_text':'text'}, inplace = True)
test_df.info()

In [ ]:
tst_dl = dls.test_dl(test_df)
tst_dl.show_batch()

In [ ]:
probs,_,idxs = learn.get_preds(dl=tst_dl, with_decoded=True)
probs

In [ ]:
# column names for probabilities
probs_df = pd.DataFrame(probs.numpy(),columns = dls.vocab[1])
probs_df

In [ ]:
probs_df["discourse_id"] = test_df["discourse_id"]
probs_df.to_csv('submission.csv', index=False)
!head submission.csv

In [ ]:
# not working for this competetion
# if not iskaggle:
#     from kaggle import api
#     api.competition_submit_cli('submission.csv', 'initial', comp)

In [ ]:
if not iskaggle:
    push_notebook('saan', comp,
                  title='Feedback effeciveness: centre context full ulmfit',
                  file='/content/drive/MyDrive/Colab Notebooks/all_text_concat_ulmfit_model.ipynb',
                  competition=comp, private=False, gpu=True)

Your kernel title does not resolve to the specified id. This may result in surprising behavior. We suggest making your title something that resolves to the specified id. See https://en.wikipedia.org/wiki/Clean_URL#Slug for more information on how slugs are determined.
Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/saansd2003/feedback-effeciveness-centre-context-full-ulmfit
